# Assembling our quarterback dataset
Along with the EDA, we also do the data wrangling to put together the dataset we'll use for our quarterback model.<br>

In [162]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import nfl_data_py as nfl
from functions import get_current_weekday, calculate_nfl_week, get_next_sunday, get_current_year

In [163]:
day = get_current_weekday()

In [164]:
date_string = get_next_sunday(day)

In [165]:
week = calculate_nfl_week(date_string)

In [166]:
season = get_current_year()

In [167]:
import sqlite3

In [168]:
# Connect to the SQLite database
#These are the FD and DK player lists for the current week
conn = sqlite3.connect('nfl_dfs.db')

query_fd = "SELECT * FROM fd_table_" + str(week) + "_" + str(season)[2:]
query_dk = "SELECT * FROM dk_table_" + str(week) + "_" + str(season)[2:]

fanduel_df = pd.read_sql_query(query_fd, conn)
draftkings_df = pd.read_sql_query(query_dk, conn)

# Close the database connection
conn.close()

In [169]:
#The weekly and play-by-play data through the previous week
#We went into R Studio to get these and save them as CSVs that we can use here
weekly_df = pd.read_csv('weekly_data_' + str(season) + '_' + str(week) + '.csv')
pbp_df = pd.read_csv('pbp_data_' + str(season) + '_' + str(week) + '.csv', low_memory = False)

In [170]:
weekly_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,1,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,10.38,10.38
1,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,2,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,9.40,9.40
2,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,3,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,14.74,14.74
3,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,4,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,25.40,25.40
4,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,5,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,19.74,19.74


In [171]:
weekly_df.tail()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
11898,00-0039919,R.Odunze,Rome Odunze,WR,WR,https://static.www.nfl.com/image/upload/f_auto...,CHI,2024,1,REG,...,1,-2.087302,0,0.500000,0.137931,0.102804,0.278859,0,1.1,2.1
11899,00-0039919,R.Odunze,Rome Odunze,WR,WR,https://static.www.nfl.com/image/upload/f_auto...,CHI,2024,2,REG,...,1,-2.121063,0,0.523810,0.147059,0.254032,0.398411,0,3.3,5.3
11900,00-0039920,M.Corley,Malachi Corley,WR,WR,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,2,REG,...,0,-0.475780,0,0.000000,0.034483,-0.006579,0.047119,0,0.4,1.4
11901,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,1,REG,...,0,-0.088616,0,-1.250000,0.032258,-0.020942,0.033727,0,1.8,2.8
11902,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,2,REG,...,0,0.238474,0,-1.166667,0.047619,-0.028037,0.051802,0,1.7,2.7


In [172]:
pbp_df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NaN,NaN,NaN,...,0,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.443521,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,1.468819,NaN,NaN,NaN,NaN,NaN,0.440373,-44.037291
3,89,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.492192,0.727261,6.988125,6.0,0.60693,0.227598,0.389904,61.009598
4,115,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.325931,NaN,NaN,NaN,NaN,NaN,0.443575,-44.357494


In [173]:
pbp_df.tail()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
104555,4210,2024_02_TB_DET,2024091503,DET,TB,REG,2,DET,home,TB,...,0,0,-0.117766,0.258139,2.288926,1.0,1.000000,0.999000,0.981842,1.815784
104556,4233,2024_02_TB_DET,2024091503,DET,TB,REG,2,DET,home,TB,...,0,0,-0.019561,0.199489,2.292825,1.0,1.000000,0.997375,0.979417,2.058339
104557,4256,2024_02_TB_DET,2024091503,DET,TB,REG,2,DET,home,TB,...,0,0,-2.230736,0.245754,2.506542,1.0,0.991035,0.991035,0.990773,0.922740
104558,4279,2024_02_TB_DET,2024091503,DET,TB,REG,2,TB,away,DET,...,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104559,4301,2024_02_TB_DET,2024091503,DET,TB,REG,2,TB,away,DET,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
qb_cols = ['player_id', 'player_name', 'player_display_name', 'position', 'recent_team', 'season', 'week',\
 'season_type', 'opponent_team', 'completions', 'attempts',\
 'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',\
 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',\
 'passing_yards_after_catch', 'passing_epa',\
 'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',\
 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_2pt_conversions',\
 'receptions', 'receiving_yards', 'receiving_tds',\
 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_2pt_conversions', 'racr', 'fantasy_points', 'fantasy_points_ppr']

# Weekly data
This weekly data will enable us to derive the fantasy points of each QB in every game.

In [175]:
weekly_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11903 entries, 0 to 11902
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    11903 non-null  object 
 1   player_name                  11903 non-null  object 
 2   player_display_name          11903 non-null  object 
 3   position                     11903 non-null  object 
 4   position_group               11903 non-null  object 
 5   headshot_url                 11853 non-null  object 
 6   recent_team                  11903 non-null  object 
 7   season                       11903 non-null  int64  
 8   week                         11903 non-null  int64  
 9   season_type                  11903 non-null  object 
 10  opponent_team                11903 non-null  object 
 11  completions                  11903 non-null  int64  
 12  attempts                     11903 non-null  int64  
 13  passing_yards   

In [176]:
pbp_df = pbp_df.replace({'LA' : 'LAR', 'JAC' : 'JAX'})

In [177]:
weekly_df = weekly_df.replace({'LA' : 'LAR', 'JAC' : 'JAX'})

In [178]:
qb_df = weekly_df[weekly_df['position'] == 'QB']

In [179]:
qb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1462 entries, 0 to 11897
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    1462 non-null   object 
 1   player_name                  1462 non-null   object 
 2   player_display_name          1462 non-null   object 
 3   position                     1462 non-null   object 
 4   position_group               1462 non-null   object 
 5   headshot_url                 1460 non-null   object 
 6   recent_team                  1462 non-null   object 
 7   season                       1462 non-null   int64  
 8   week                         1462 non-null   int64  
 9   season_type                  1462 non-null   object 
 10  opponent_team                1462 non-null   object 
 11  completions                  1462 non-null   int64  
 12  attempts                     1462 non-null   int64  
 13  passing_yards         

In [180]:
qb_df = qb_df[qb_cols]

In [181]:
qb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1462 entries, 0 to 11897
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   player_id                  1462 non-null   object 
 1   player_name                1462 non-null   object 
 2   player_display_name        1462 non-null   object 
 3   position                   1462 non-null   object 
 4   recent_team                1462 non-null   object 
 5   season                     1462 non-null   int64  
 6   week                       1462 non-null   int64  
 7   season_type                1462 non-null   object 
 8   opponent_team              1462 non-null   object 
 9   completions                1462 non-null   int64  
 10  attempts                   1462 non-null   int64  
 11  passing_yards              1462 non-null   int64  
 12  passing_tds                1462 non-null   int64  
 13  interceptions              1462 non-null   int64  
 

In [182]:
fanduel_df = fanduel_df[fanduel_df['position'] == 'QB']
draftkings_df = draftkings_df[draftkings_df['position'] == 'QB']

# Setting up for concatenation
We need to add the current week's player list to the data that goes through last week so we can derive L8 variables for the current week. There are no DraftKings names that aren't in the FanDuel list, so we can just use the FanDuel list to concat.

In [183]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week
4,107027-63115,Lamar Jackson,QB,8800,BAL,DAL,0,Active,09-22-2024,3
5,107027-57439,Patrick Mahomes,QB,8700,KC,ATL,0,Active,09-22-2024,3
9,107027-69531,Jalen Hurts,QB,8500,PHI,NO,0,Active,09-22-2024,3
13,107027-63484,Kyler Murray,QB,8300,ARI,DET,1,Active,09-22-2024,3
15,107027-129471,C.J. Stroud,QB,8200,HOU,MIN,0,Active,09-22-2024,3


In [184]:
fanduel_df.replace({'Gardner Minshew II': 'Gardner Minshew', 'PJ Walker': 'P.J. Walker'}, inplace = True)
draftkings_df.replace({'Gardner Minshew II': 'Gardner Minshew', 'PJ Walker': 'P.J. Walker'}, inplace = True)

In [185]:
new_df = fanduel_df[['name', 'position', 'team', 'opponent', 'week']]
new_df.rename(columns = {'name': 'player_display_name', 'team' : 'recent_team', 'opponent': 'opponent_team'}, inplace = True)
new_df['season'] = 2024

C:\Users\Owner\AppData\Local\Temp\ipykernel_21300\2074499718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.rename(columns = {'name': 'player_display_name', 'team' : 'recent_team', 'opponent': 'opponent_team'}, inplace = True)
C:\Users\Owner\AppData\Local\Temp\ipykernel_21300\2074499718.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['season'] = 2024


In [186]:
qb_df.drop(columns = ['player_id', 'player_name', 'season_type'], inplace = True)

In [187]:
qb_df = pd.concat([qb_df, new_df], axis = 0)

In [188]:
qb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1569 entries, 0 to 178
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   player_display_name        1569 non-null   object 
 1   position                   1569 non-null   object 
 2   recent_team                1569 non-null   object 
 3   season                     1569 non-null   int64  
 4   week                       1569 non-null   int64  
 5   opponent_team              1569 non-null   object 
 6   completions                1462 non-null   float64
 7   attempts                   1462 non-null   float64
 8   passing_yards              1462 non-null   float64
 9   passing_tds                1462 non-null   float64
 10  interceptions              1462 non-null   float64
 11  sacks                      1462 non-null   float64
 12  sack_yards                 1462 non-null   float64
 13  sack_fumbles               1462 non-null   float64
 14

In [189]:
qb_cols_to_group = ['season', 'week', 'player_display_name', 'recent_team', 'opponent_team']

In [190]:
qb_scoring_cols = ['passing_yards', 'passing_tds', 'interceptions', 'passing_2pt_conversions', 'rushing_yards',\
                   'rushing_tds', 'rushing_fumbles_lost', 'rushing_2pt_conversions', 'receptions', 'receiving_yards',\
                   'receiving_tds', 'receiving_fumbles_lost', 'receiving_2pt_conversions', 'sack_fumbles_lost' ]

In [191]:
qb_df = qb_df.set_index(qb_cols_to_group, drop = True)

# Fantasy points
This is where we calculate FanDuel and DraftKings points.

In [192]:
qb_df['FD_Pts'] = (qb_df['passing_yards'] * 0.04) + (qb_df['rushing_tds'] * 6) + (qb_df['rushing_yards'] * 0.1)\
+ (qb_df['passing_tds'] * 4) + (qb_df['receiving_yards'] * 0.1) + (qb_df['receiving_tds'] * 6) + (qb_df['receptions'] * 0.5)\
+ (qb_df['rushing_2pt_conversions'] * 2) + (qb_df['passing_2pt_conversions'] * 2) + (qb_df['receiving_2pt_conversions'] * 2)\
- (qb_df['interceptions']) - (qb_df['sack_fumbles_lost'] * 2) - (qb_df['receiving_fumbles_lost'] * 2) - (qb_df['rushing_fumbles_lost'] * 2)

In [193]:
qb_df['DK_Pts'] = (qb_df['passing_yards'] * 0.04) + (qb_df['rushing_tds'] * 6) + (qb_df['rushing_yards'] * 0.1)\
+ (qb_df['passing_tds'] * 4) + (qb_df['receiving_yards'] * 0.1) + (qb_df['receiving_tds'] * 6) + (qb_df['receptions'] * 1)\
+ (qb_df['rushing_2pt_conversions'] * 2) + (qb_df['passing_2pt_conversions'] * 2) + (qb_df['receiving_2pt_conversions'] * 2)\
- (qb_df['interceptions']) - (qb_df['sack_fumbles_lost']) - (qb_df['receiving_fumbles_lost']) - (qb_df['rushing_fumbles_lost'])


Adding DraftKings bonus points

In [194]:
qb_df['DK_Pts'] = np.where(qb_df['passing_yards'] >= 300, qb_df['DK_Pts'] + 3, qb_df['DK_Pts'])
qb_df['DK_Pts'] = np.where(qb_df['receiving_yards'] >= 100, qb_df['DK_Pts'] + 3, qb_df['DK_Pts'])
qb_df['DK_Pts'] = np.where(qb_df['rushing_yards'] >= 100, qb_df['DK_Pts'] + 3, qb_df['DK_Pts'])

In [195]:
#pacr = passing air conversion ratio (passing yards/air yards)

In [196]:
qb_df.reset_index(inplace = True)

In [197]:
qb_df.tail()

,season,week,player_display_name,recent_team,opponent_team,position,completions,attempts,passing_yards,passing_tds,...,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_2pt_conversions,racr,fantasy_points,fantasy_points_ppr,FD_Pts,DK_Pts
1564,2024,3,Blaine Gabbert,KC,ATL,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1565,2024,3,Kenny Pickett,PHI,NO,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1566,2024,3,Andy Dalton,CAR,LV,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1567,2024,3,Hendon Hooker,DET,ARI,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1568,2024,3,Jason Bean,IND,CHI,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
#fanduel_df.head()

In [199]:
#draftkings_df.head()

In [200]:
#We need to group QBs by team because for now we're trying to get DvP means for teams against QBs
grouped_pts = qb_df.groupby(['season', 'week', 'recent_team', 'opponent_team'])[['DK_Pts', 'FD_Pts']].sum().round(3)

In [201]:
#grouped_pts[(grouped_pts['opponent_team'] == 'CLE') & (grouped_pts['season'] == 2001) & (grouped_pts['week'] == 11)]

In [202]:
grouped_pts.reset_index(inplace = True)

In [203]:
grouped_pts

,season,week,recent_team,opponent_team,DK_Pts,FD_Pts
0,2022,1,ARI,KC,23.10,23.10
1,2022,1,ATL,NO,20.80,19.80
2,2022,1,BAL,NYJ,21.22,21.22
3,2022,1,BUF,LAR,33.48,33.48
4,2022,1,CAR,CLE,19.00,19.00
...,...,...,...,...,...,...
1223,2024,3,PIT,LAC,0.00,0.00
1224,2024,3,SEA,MIA,0.00,0.00
1225,2024,3,SF,LAR,0.00,0.00
1226,2024,3,TB,DEN,0.00,0.00


In [204]:
grouped_pts = grouped_pts.sort_values(by = ['opponent_team', 'season', 'week'])
grouped_pts['opp_game_num'] = grouped_pts.groupby(['opponent_team']).cumcount() + 1
grouped_pts.reset_index(inplace = True)

In [205]:
grouped_pts_raw = grouped_pts.copy()

In [206]:
grouped_pts.drop(columns = ['index'], inplace = True)

# DvP variables
We'll take an 8-game rolling mean for fantasy points allowed to QBs for each team, even if the games go back to last season. DvP means defense vs. position.

In [207]:
def calculate_equal_rolling_mean(group, cols, suffix):
    """
    This function calculates a rolling mean for the last eight games, going back to previous season if necessary.
    It also calculates when there are less than eight games to use.
    """
    for col in cols:
        group[f'{col}{suffix}'] = (
            group[col].shift().rolling(window=8, min_periods=1).mean()
        )
    return group

In [208]:
grouped_pts = grouped_pts.groupby('opponent_team', as_index = False).apply(calculate_equal_rolling_mean, cols=['FD_Pts', 'DK_Pts'], suffix = '_DvP')

In [209]:
grouped_pts.drop(columns = ['DK_Pts', 'FD_Pts', 'opp_game_num'], inplace = True)

In [210]:
qb_df = pd.merge(qb_df, grouped_pts, on = ['season', 'week', 'recent_team', 'opponent_team'], how = 'left')

In [211]:
qb_df.head()

,season,week,player_display_name,recent_team,opponent_team,position,completions,attempts,passing_yards,passing_tds,...,receiving_fumbles,receiving_fumbles_lost,receiving_2pt_conversions,racr,fantasy_points,fantasy_points_ppr,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP
0,2022,1,Tom Brady,TB,DAL,QB,18.0,27.0,212.0,1.0,...,0.0,0.0,0.0,NaN,10.38,10.38,11.38,11.38,NaN,NaN
1,2022,2,Tom Brady,TB,NO,QB,18.0,34.0,190.0,1.0,...,0.0,0.0,0.0,NaN,9.40,9.40,9.40,10.40,19.800000,20.800
2,2022,3,Tom Brady,TB,GB,QB,31.0,42.0,271.0,1.0,...,0.0,0.0,0.0,NaN,14.74,14.74,14.74,14.74,14.440000,14.440
3,2022,4,Tom Brady,TB,KC,QB,39.0,52.0,385.0,3.0,...,0.0,0.0,0.0,NaN,25.40,25.40,25.40,29.40,20.946667,22.280
4,2022,5,Tom Brady,TB,ATL,QB,35.0,52.0,351.0,1.0,...,0.0,0.0,0.0,NaN,19.74,19.74,19.74,22.74,23.300000,24.175


In [212]:
qb_points_last_week = qb_df[(qb_df['season'] == season) & (qb_df['week'] == week - 1)]\
[['season', 'week', 'player_display_name', 'recent_team', 'opponent_team', 'position', 'FD_Pts', 'DK_Pts']]

In [213]:
qb_points_last_week.to_csv('qb_points_' + str(season) + '_' + str(week - 1) + '.csv')

In [214]:
# conn = sqlite3.connect('nfl_dfs.db')

# # Specify the table name
# table_name = 'pbp_non_defense'

# # Query the table and load it into a pandas DataFrame
# pbp_df = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)

# # Display the DataFrame
# print(pbp_df)

# # Close the connection
# conn.close()

In [215]:
pbp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104560 entries, 0 to 104559
Columns: 372 entries, play_id to pass_oe
dtypes: float64(182), int64(39), object(151)
memory usage: 296.8+ MB


In [216]:
qb_df.reset_index(inplace = True)

In [217]:
qb_df.rename(columns = {'recent_team': 'posteam'}, inplace = True)

# Introducing role variables
We brought in play-by-play data to derive our role variable. For games where more than one quarterback took a snap for a team, we're going to see which one took more snaps and give that QB a QB_role value of 1. All other QBs will have a value of 2. This technique will take on greater importance when we do RBs and WRs. In the case of QBs, if a backup quarterback becomes the starter, we'll change the role value to 1, and that should provide a prediction that's more reflective of the QBs greater role.<br>

In [218]:
# Assuming passer_df is already created
passer_df = pbp_df.groupby(['season', 'week', 'posteam', 'passer_player_id'])['play_id'].count().reset_index()

In [219]:
# Sort by season, week, posteam, and the number of plays in descending order
passer_df = passer_df.sort_values(by=['season', 'week', 'posteam', 'play_id'], ascending=[True, True, True, False])

In [220]:
# Create a rank column to identify the top two QBs
passer_df['QB_role'] = passer_df.groupby(['season', 'week', 'posteam'])['play_id'].rank(method='first', ascending=False)

In [221]:
# Assign 'QB1' or 'QB2' based on the rank
#passer_df['QB_role'] = passer_df['rank'].apply(lambda x: 'QB1' if x == 1 else ('QB2' if x == 2 else None))

In [222]:
# Drop the rank column if you don't need it anymore
#passer_df = passer_df.drop(columns=['rank'])

In [223]:
#passer_df.rename(columns = {'passer_player_id': 'player_id'}, inplace = True)

In [224]:
passer_df.drop(columns = ['play_id'], inplace = True)

In [225]:
passer_df.head()

,season,week,posteam,passer_player_id,QB_role
1,2022,1,ARI,00-0035228,1.0
0,2022,1,ARI,00-0035146,2.0
2,2022,1,ATL,00-0032268,1.0
3,2022,1,BAL,00-0034796,1.0
4,2022,1,BUF,00-0034857,1.0


In [226]:
passer_df.rename(columns = {'passer_player_id': 'player_id', 'posteam': 'recent_team'}, inplace = True)

We need to merge what we get from the play-by-play data back into the weekly data since the weekly data uses full names, and the pbp only uses first initial and last name.

In [227]:
merge_df = weekly_df[weekly_df['position'] == 'QB']

In [228]:
merge_df = merge_df[['player_id', 'season', 'week', 'recent_team', 'player_display_name']]

In [229]:
merge_df = pd.merge(merge_df, passer_df, on = ['season', 'week', 'recent_team', 'player_id'], how = 'left')

In [230]:
merge_df

,player_id,season,week,recent_team,player_display_name,QB_role
0,00-0019596,2022,1,TB,Tom Brady,1.0
1,00-0019596,2022,2,TB,Tom Brady,1.0
2,00-0019596,2022,3,TB,Tom Brady,1.0
3,00-0019596,2022,4,TB,Tom Brady,1.0
4,00-0019596,2022,5,TB,Tom Brady,1.0
...,...,...,...,...,...,...
1457,00-0039732,2024,2,DEN,Bo Nix,1.0
1458,00-0039910,2024,1,WAS,Jayden Daniels,1.0
1459,00-0039910,2024,2,WAS,Jayden Daniels,1.0
1460,00-0039918,2024,1,CHI,Caleb Williams,1.0


In [231]:
merge_df.rename(columns = {'recent_team' : 'posteam'}, inplace = True)
#merge_df.drop(columns = ['player_id'], inplace = True)

# Merging
Now we'll merge so that our QB_role variable is added to the dataframe. This is where we also change the name of the dataframe to **quarterback_df.**

In [232]:
quarterback_df = pd.merge(qb_df, merge_df, on = ['season', 'week', 'posteam', 'player_display_name'], how = 'left')

In [233]:
quarterback_df.tail()

,index,season,week,player_display_name,posteam,opponent_team,position,completions,attempts,passing_yards,...,receiving_2pt_conversions,racr,fantasy_points,fantasy_points_ppr,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP,player_id,QB_role
1564,1564,2024,3,Blaine Gabbert,KC,ATL,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.9225,18.2975,NaN,NaN
1565,1565,2024,3,Kenny Pickett,PHI,NO,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.7625,15.7625,NaN,NaN
1566,1566,2024,3,Andy Dalton,CAR,LV,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.9900,15.2400,NaN,NaN
1567,1567,2024,3,Hendon Hooker,DET,ARI,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20.5975,20.9725,NaN,NaN
1568,1568,2024,3,Jason Bean,IND,CHI,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.3400,15.4650,NaN,NaN


In [234]:
#quarterback_df = quarterback_df.dropna(subset=['QB_role'])

# Paring down some of the columns
We're drop columns that won't be needed for features.

In [235]:
cols_to_keep = ['season', 'week', 'player_id', 'player_display_name', 'posteam',
       'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds',
       'interceptions', 'sacks', 'sack_fumbles_lost', 'passing_air_yards', 
        'pacr',  'carries', 'rushing_yards', 'rushing_tds', 'FD_Pts', 'DK_Pts', 'FD_Pts_DvP', 'DK_Pts_DvP', 'QB_role']

In [236]:
quarterback_df = quarterback_df[cols_to_keep]

In [237]:
#quarterback_df[quarterback_df['season'] >= 2006]['passing_air_yards']

In [238]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1569 entries, 0 to 1568
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1569 non-null   int64  
 1   week                 1569 non-null   int64  
 2   player_id            1462 non-null   object 
 3   player_display_name  1569 non-null   object 
 4   posteam              1569 non-null   object 
 5   opponent_team        1569 non-null   object 
 6   completions          1462 non-null   float64
 7   attempts             1462 non-null   float64
 8   passing_yards        1462 non-null   float64
 9   passing_tds          1462 non-null   float64
 10  interceptions        1462 non-null   float64
 11  sacks                1462 non-null   float64
 12  sack_fumbles_lost    1462 non-null   float64
 13  passing_air_yards    1462 non-null   float64
 14  pacr                 1393 non-null   float64
 15  carries              1462 non-null   f

# Checking missing on previous weeks
Before we check missing values, we should temporarily break up the dataframe so that the current week, which we're trying to fill, is separate from the rest of the data. Then we can deal with missing values for previous weeks before bringing back the rows we want to fill.

In [239]:
qb_df_temp = quarterback_df[~((quarterback_df['season'] == 2024) & (quarterback_df['week'] == 2))]

In [240]:
this_week = quarterback_df[(quarterback_df['season'] == 2024) & (quarterback_df['week'] == 2)]

In [241]:
qb_df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1530 entries, 0 to 1568
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1530 non-null   int64  
 1   week                 1530 non-null   int64  
 2   player_id            1423 non-null   object 
 3   player_display_name  1530 non-null   object 
 4   posteam              1530 non-null   object 
 5   opponent_team        1530 non-null   object 
 6   completions          1423 non-null   float64
 7   attempts             1423 non-null   float64
 8   passing_yards        1423 non-null   float64
 9   passing_tds          1423 non-null   float64
 10  interceptions        1423 non-null   float64
 11  sacks                1423 non-null   float64
 12  sack_fumbles_lost    1423 non-null   float64
 13  passing_air_yards    1423 non-null   float64
 14  pacr                 1359 non-null   float64
 15  carries              1423 non-null   float6

In [242]:
this_week.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 1384 to 1461
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               39 non-null     int64  
 1   week                 39 non-null     int64  
 2   player_id            39 non-null     object 
 3   player_display_name  39 non-null     object 
 4   posteam              39 non-null     object 
 5   opponent_team        39 non-null     object 
 6   completions          39 non-null     float64
 7   attempts             39 non-null     float64
 8   passing_yards        39 non-null     float64
 9   passing_tds          39 non-null     float64
 10  interceptions        39 non-null     float64
 11  sacks                39 non-null     float64
 12  sack_fumbles_lost    39 non-null     float64
 13  passing_air_yards    39 non-null     float64
 14  pacr                 34 non-null     float64
 15  carries              39 non-null     float

In [243]:
# quarterback_df['passing_air_yards'] = np.where((quarterback_df['season'] <= 2005) & (quarterback_df['QB_role'] == 1),\
#                                                quarterback_df['passing_air_yards'].mean(), quarterback_df['passing_air_yards'])

# Filling pacr
pacr is passing yards/air yards, and every QB will have at least some air yards even if he doesn't have passing yards. So we were going to just fill missing pacr values with 0, but upon further review, we thought about filling with 1. Then we saw the mean is almost 1 (0.93) and the median is .86. So we'll just fill with the mean

In [244]:
qb_df_temp['pacr'] = qb_df_temp['pacr'].fillna(qb_df_temp['pacr'].mean())

C:\Users\Owner\AppData\Local\Temp\ipykernel_21300\52539329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qb_df_temp['pacr'] = qb_df_temp['pacr'].fillna(qb_df_temp['pacr'].mean())


Let's see if dropping all 2023 Week 1 rows will take care of some of the missing values. We obviously don't need that for L8 variables.

In [245]:
# quarterback_df['FD_Pts_DvP'] = quarterback_df['FD_Pts_DvP'].fillna(quarterback_df['FD_Pts_DvP'].mean())
# quarterback_df['DK_Pts_DvP'] = quarterback_df['DK_Pts_DvP'].fillna(quarterback_df['DK_Pts_DvP'].mean())
qb_df_temp = qb_df_temp[~((qb_df_temp['season'] == 2023) & (qb_df_temp['week'] == 1))]

In [246]:
qb_df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1490 entries, 0 to 1568
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1490 non-null   int64  
 1   week                 1490 non-null   int64  
 2   player_id            1383 non-null   object 
 3   player_display_name  1490 non-null   object 
 4   posteam              1490 non-null   object 
 5   opponent_team        1490 non-null   object 
 6   completions          1383 non-null   float64
 7   attempts             1383 non-null   float64
 8   passing_yards        1383 non-null   float64
 9   passing_tds          1383 non-null   float64
 10  interceptions        1383 non-null   float64
 11  sacks                1383 non-null   float64
 12  sack_fumbles_lost    1383 non-null   float64
 13  passing_air_yards    1383 non-null   float64
 14  pacr                 1490 non-null   float64
 15  carries              1383 non-null   float6

In [247]:
this_week.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 1384 to 1461
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               39 non-null     int64  
 1   week                 39 non-null     int64  
 2   player_id            39 non-null     object 
 3   player_display_name  39 non-null     object 
 4   posteam              39 non-null     object 
 5   opponent_team        39 non-null     object 
 6   completions          39 non-null     float64
 7   attempts             39 non-null     float64
 8   passing_yards        39 non-null     float64
 9   passing_tds          39 non-null     float64
 10  interceptions        39 non-null     float64
 11  sacks                39 non-null     float64
 12  sack_fumbles_lost    39 non-null     float64
 13  passing_air_yards    39 non-null     float64
 14  pacr                 34 non-null     float64
 15  carries              39 non-null     float

In [248]:
quarterback_df = pd.concat([qb_df_temp, this_week], axis = 0)

In [249]:
#With the dataframes back together, we can universally fill in missing QB_role values with 2
quarterback_df['QB_role'] = quarterback_df['QB_role'].fillna(2)

In [250]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1529 entries, 0 to 1461
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1529 non-null   int64  
 1   week                 1529 non-null   int64  
 2   player_id            1422 non-null   object 
 3   player_display_name  1529 non-null   object 
 4   posteam              1529 non-null   object 
 5   opponent_team        1529 non-null   object 
 6   completions          1422 non-null   float64
 7   attempts             1422 non-null   float64
 8   passing_yards        1422 non-null   float64
 9   passing_tds          1422 non-null   float64
 10  interceptions        1422 non-null   float64
 11  sacks                1422 non-null   float64
 12  sack_fumbles_lost    1422 non-null   float64
 13  passing_air_yards    1422 non-null   float64
 14  pacr                 1524 non-null   float64
 15  carries              1422 non-null   float6

In [251]:
quarterback_df.drop(columns = ['player_id'], inplace = True)

In [252]:
missing_df = quarterback_df[quarterback_df.isna().any(axis = 1)]

In [253]:
missing_df['season'].value_counts()

season
2024    112
2022     37
Name: count, dtype: int64

In [254]:
missing_df['week'].value_counts()

week
3    107
1     37
2      5
Name: count, dtype: int64

In [255]:
cols_L8 = ['completions', 'attempts', 'passing_yards', 'passing_tds','interceptions',\
           'sacks', 'passing_air_yards', 'pacr', 'carries', 'rushing_yards', 'rushing_tds']

In [256]:
qb_L8_features = quarterback_df.groupby(['player_display_name', 'season', 'week'])[cols_L8].sum()

In [257]:
qb_L8_features

completions  attempts  passing_yards  \
player_display_name season week                                         
AJ McCarron         2023   14            1.0       1.0           -1.0   
                           18            3.0       4.0           20.0   
Aaron Rodgers       2022   1            22.0      34.0          195.0   
                           2            19.0      25.0          234.0   
                           3            27.0      35.0          255.0   
...                                      ...       ...            ...   
Zach Wilson         2023   11            7.0      15.0           81.0   
                           14           27.0      36.0          301.0   
                           15            4.0      11.0           26.0   
                    2024   3             0.0       0.0            0.0   
Zack Annexstad      2024   3             0.0       0.0            0.0   

                                 passing_tds  interceptions  sacks  \
player_display_name season week                                      
AJ McCarron         2023   14            0.0            0.0    0.0   
                           18            0.0            0.0    1.0   
Aaron Rodgers       2022   1             0.0            1.0    4.0   
                           2             2.0            0.0    3.0   
                           3             2.0            1.0    1.0   
...                                      ...            ...    ...   
Zach Wilson         2023   11            1.0            1.0    5.0   
                           14            2.0            0.0    4.0   
                           15            0.0            0.0    4.0   
                    2024   3             0.0            0.0    0.0   
Zack Annexstad      2024   3             0.0            0.0    0.0   

                                 passing_air_yards      pacr  carries  \
player_display_name season week                                         
AJ McCarron         2023   14                 -1.0  0.000000      0.0   
                           18                 -7.0  0.000000      0.0   
Aaron Rodgers       2022   1                 190.0  1.026316      1.0   
                           2                 139.0  1.683453      5.0   
                           3                 177.0  1.440678      1.0   
...                                            ...       ...      ...   
Zach Wilson         2023   11                 76.0  1.065789      4.0   
                           14                238.0  1.264706      3.0   
                           15                 29.0  0.896552      0.0   
                    2024   3                   0.0  0.966720      0.0   
Zack Annexstad      2024   3                   0.0  0.966720      0.0   

                                 rushing_yards  rushing_tds  
player_display_name season week                              
AJ McCarron         2023   14              0.0          0.0  
                           18              0.0          0.0  
Aaron Rodgers       2022   1              -1.0          0.0  
                           2              10.0          0.0  
                           3              -1.0          0.0  
...                                        ...          ...  
Zach Wilson         2023   11             15.0          0.0  
                           14             12.0          0.0  
                           15              0.0          0.0  
                    2024   3               0.0          0.0  
Zack Annexstad      2024   3               0.0          0.0  

[1529 rows x 11 columns]

# L8 variables
L8 variables are rolling means of features over the last eight games that each QB has played. Just like we did for the DvP variables, we'll calculate features over the previous eight games for individual QBs.

In [258]:
qb_L8_features = qb_L8_features.sort_values(by = ['player_display_name', 'season', 'week'])
qb_L8_features['game_num'] = qb_L8_features.groupby(['player_display_name', 'season']).cumcount() + 1
#quarterback_df.reset_index(drop = True, inplace = True)

In [259]:
qb_L8_features

completions  attempts  passing_yards  \
player_display_name season week                                         
AJ McCarron         2023   14            1.0       1.0           -1.0   
                           18            3.0       4.0           20.0   
Aaron Rodgers       2022   1            22.0      34.0          195.0   
                           2            19.0      25.0          234.0   
                           3            27.0      35.0          255.0   
...                                      ...       ...            ...   
Zach Wilson         2023   11            7.0      15.0           81.0   
                           14           27.0      36.0          301.0   
                           15            4.0      11.0           26.0   
                    2024   3             0.0       0.0            0.0   
Zack Annexstad      2024   3             0.0       0.0            0.0   

                                 passing_tds  interceptions  sacks  \
player_display_name season week                                      
AJ McCarron         2023   14            0.0            0.0    0.0   
                           18            0.0            0.0    1.0   
Aaron Rodgers       2022   1             0.0            1.0    4.0   
                           2             2.0            0.0    3.0   
                           3             2.0            1.0    1.0   
...                                      ...            ...    ...   
Zach Wilson         2023   11            1.0            1.0    5.0   
                           14            2.0            0.0    4.0   
                           15            0.0            0.0    4.0   
                    2024   3             0.0            0.0    0.0   
Zack Annexstad      2024   3             0.0            0.0    0.0   

                                 passing_air_yards      pacr  carries  \
player_display_name season week                                         
AJ McCarron         2023   14                 -1.0  0.000000      0.0   
                           18                 -7.0  0.000000      0.0   
Aaron Rodgers       2022   1                 190.0  1.026316      1.0   
                           2                 139.0  1.683453      5.0   
                           3                 177.0  1.440678      1.0   
...                                            ...       ...      ...   
Zach Wilson         2023   11                 76.0  1.065789      4.0   
                           14                238.0  1.264706      3.0   
                           15                 29.0  0.896552      0.0   
                    2024   3                   0.0  0.966720      0.0   
Zack Annexstad      2024   3                   0.0  0.966720      0.0   

                                 rushing_yards  rushing_tds  game_num  
player_display_name season week                                        
AJ McCarron         2023   14              0.0          0.0         1  
                           18              0.0          0.0         2  
Aaron Rodgers       2022   1              -1.0          0.0         1  
                           2              10.0          0.0         2  
                           3              -1.0          0.0         3  
...                                        ...          ...       ...  
Zach Wilson         2023   11             15.0          0.0         9  
                           14             12.0          0.0        10  
                           15              0.0          0.0        11  
                    2024   3               0.0          0.0         1  
Zack Annexstad      2024   3               0.0          0.0         1  

[1529 rows x 12 columns]

In [260]:
qb_L8_features = qb_L8_features.groupby(['player_display_name'], as_index = False).apply(calculate_equal_rolling_mean, cols=cols_L8, suffix = '_L8')

In [261]:
qb_L8_features.reset_index(inplace = True)

In [262]:
qb_L8_features

,level_0,player_display_name,season,week,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,attempts_L8,passing_yards_L8,passing_tds_L8,interceptions_L8,sacks_L8,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8
0,0,AJ McCarron,2023,14,1.0,1.0,-1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,AJ McCarron,2023,18,3.0,4.0,20.0,0.0,0.0,1.0,...,1.000,-1.000,0.00,0.000,0.000,-1.000,0.000000,0.000,0.000,0.0
2,1,Aaron Rodgers,2022,1,22.0,34.0,195.0,0.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Aaron Rodgers,2022,2,19.0,25.0,234.0,2.0,0.0,3.0,...,34.000,195.000,0.00,1.000,4.000,190.000,1.026316,1.000,-1.000,0.0
4,1,Aaron Rodgers,2022,3,27.0,35.0,255.0,2.0,1.0,1.0,...,29.500,214.500,1.00,0.500,3.500,164.500,1.354885,3.000,4.500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1524,137,Zach Wilson,2023,11,7.0,15.0,81.0,1.0,1.0,5.0,...,35.625,215.375,0.50,0.625,3.875,279.125,0.785091,3.125,22.250,0.0
1525,137,Zach Wilson,2023,14,27.0,36.0,301.0,2.0,0.0,4.0,...,34.125,204.250,0.50,0.375,4.125,254.500,0.840476,3.000,19.625,0.0
1526,137,Zach Wilson,2023,15,4.0,11.0,26.0,0.0,0.0,4.0,...,34.125,222.250,0.75,0.375,4.250,253.875,0.917803,3.250,21.000,0.0
1527,137,Zach Wilson,2024,3,0.0,0.0,0.0,0.0,0.0,0.0,...,30.625,194.875,0.50,0.375,4.500,221.125,0.924631,3.000,19.250,0.0


In [263]:
qb_L8_features.drop(columns = ['level_0', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',\
                              'sacks', 'passing_air_yards', 'pacr', 'carries', 'rushing_yards', 'rushing_tds'], inplace = True)

In [264]:
quarterback_df = pd.merge(quarterback_df, qb_L8_features, on = ['player_display_name', 'season', 'week'], how = 'left')

In [265]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1529 entries, 0 to 1528
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1529 non-null   int64  
 1   week                  1529 non-null   int64  
 2   player_display_name   1529 non-null   object 
 3   posteam               1529 non-null   object 
 4   opponent_team         1529 non-null   object 
 5   completions           1422 non-null   float64
 6   attempts              1422 non-null   float64
 7   passing_yards         1422 non-null   float64
 8   passing_tds           1422 non-null   float64
 9   interceptions         1422 non-null   float64
 10  sacks                 1422 non-null   float64
 11  sack_fumbles_lost     1422 non-null   float64
 12  passing_air_yards     1422 non-null   float64
 13  pacr                  1524 non-null   float64
 14  carries               1422 non-null   float64
 15  rushing_yards        

In [266]:
quarterback_df[(quarterback_df.isna().any(axis = 1))]

,season,week,player_display_name,posteam,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,...,attempts_L8,passing_yards_L8,passing_tds_L8,interceptions_L8,sacks_L8,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8
0,2022,1,Tom Brady,TB,DAL,18.0,27.0,212.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,2022,1,Aaron Rodgers,GB,MIN,22.0,34.0,195.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,2022,1,Matt Ryan,IND,HOU,32.0,50.0,352.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,2022,1,Joe Flacco,NYJ,BAL,37.0,59.0,309.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,2022,7,Chad Henne,KC,SF,0.0,2.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2024,2,Jameis Winston,CLE,JAX,0.0,0.0,0.0,0.0,0.0,...,16.000,106.625000,0.50,1.000000,1.125000,204.000000,0.627707,1.000000,0.125,0.000000
1499,2024,2,Taysom Hill,NO,DAL,0.0,0.0,0.0,0.0,0.0,...,0.625,1.375000,0.00,0.000000,0.125000,7.125000,0.604200,4.625000,22.625,0.125000
1502,2024,2,Mitchell Trubisky,BUF,MIA,0.0,0.0,0.0,0.0,0.0,...,21.375,153.875000,0.75,1.000000,1.250000,197.125000,0.813268,3.500000,8.500,0.375000
1520,2024,2,Clayton Tune,ARI,LAR,0.0,0.0,0.0,0.0,0.0,...,3.500,10.333333,0.00,0.333333,1.166667,8.833333,1.008426,1.333333,5.000,0.166667


In [267]:
quarterback_df.drop(columns = ['completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',\
                              'sacks', 'passing_air_yards', 'pacr', 'carries', 'rushing_yards', 'rushing_tds',\
                              'sack_fumbles_lost', 'game_num'], inplace = True)

In [268]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1529 entries, 0 to 1528
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1529 non-null   int64  
 1   week                  1529 non-null   int64  
 2   player_display_name   1529 non-null   object 
 3   posteam               1529 non-null   object 
 4   opponent_team         1529 non-null   object 
 5   FD_Pts                1422 non-null   float64
 6   DK_Pts                1422 non-null   float64
 7   FD_Pts_DvP            1492 non-null   float64
 8   DK_Pts_DvP            1492 non-null   float64
 9   QB_role               1529 non-null   float64
 10  completions_L8        1390 non-null   float64
 11  attempts_L8           1390 non-null   float64
 12  passing_yards_L8      1390 non-null   float64
 13  passing_tds_L8        1390 non-null   float64
 14  interceptions_L8      1390 non-null   float64
 15  sacks_L8             

In [269]:
cols_to_check = ['completions_L8', 'attempts_L8', 'passing_yards_L8', 'passing_tds_L8',\
                 'interceptions_L8', 'sacks_L8', 'passing_air_yards_L8', 'pacr_L8',\
                 'carries_L8', 'rushing_yards_L8', 'rushing_tds_L8']

In [270]:
missing_L8 = quarterback_df[quarterback_df[cols_to_check].isna().any(axis = 1)]

In [271]:
#pd.set_option('display.max_rows', None)

In [272]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1529 entries, 0 to 1528
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1529 non-null   int64  
 1   week                  1529 non-null   int64  
 2   player_display_name   1529 non-null   object 
 3   posteam               1529 non-null   object 
 4   opponent_team         1529 non-null   object 
 5   FD_Pts                1422 non-null   float64
 6   DK_Pts                1422 non-null   float64
 7   FD_Pts_DvP            1492 non-null   float64
 8   DK_Pts_DvP            1492 non-null   float64
 9   QB_role               1529 non-null   float64
 10  completions_L8        1390 non-null   float64
 11  attempts_L8           1390 non-null   float64
 12  passing_yards_L8      1390 non-null   float64
 13  passing_tds_L8        1390 non-null   float64
 14  interceptions_L8      1390 non-null   float64
 15  sacks_L8             

# Must fill missing values
We need to run the new data through the model, so we can't have missing data. We'll fill with the mean.

In [273]:
columns_to_fill = [
    'completions_L8', 'attempts_L8', 'passing_yards_L8', 'passing_tds_L8',
    'interceptions_L8', 'sacks_L8', 'passing_air_yards_L8', 'pacr_L8',
    'carries_L8', 'rushing_yards_L8', 'rushing_tds_L8'
]

In [274]:
quarterback_df[columns_to_fill] = quarterback_df[columns_to_fill].apply(lambda col: col.fillna(col.mean()))

In [275]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1529 entries, 0 to 1528
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1529 non-null   int64  
 1   week                  1529 non-null   int64  
 2   player_display_name   1529 non-null   object 
 3   posteam               1529 non-null   object 
 4   opponent_team         1529 non-null   object 
 5   FD_Pts                1422 non-null   float64
 6   DK_Pts                1422 non-null   float64
 7   FD_Pts_DvP            1492 non-null   float64
 8   DK_Pts_DvP            1492 non-null   float64
 9   QB_role               1529 non-null   float64
 10  completions_L8        1529 non-null   float64
 11  attempts_L8           1529 non-null   float64
 12  passing_yards_L8      1529 non-null   float64
 13  passing_tds_L8        1529 non-null   float64
 14  interceptions_L8      1529 non-null   float64
 15  sacks_L8             

# Percentage variables
We'll take our L8 means and derive certain percentages.

In [276]:
quarterback_df['comp_pct'] = quarterback_df['completions_L8']/quarterback_df['attempts_L8']
quarterback_df['yds_per_attempt'] = quarterback_df['passing_yards_L8']/quarterback_df['attempts_L8']
quarterback_df['td_pct'] = quarterback_df['passing_tds_L8']/quarterback_df['attempts_L8']
quarterback_df['sack_pct'] = quarterback_df['sacks_L8']/quarterback_df['attempts_L8']
quarterback_df['int_pct'] = quarterback_df['interceptions_L8']/quarterback_df['attempts_L8']

In [277]:
quarterback_df.tail(10)

,season,week,player_display_name,posteam,opponent_team,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP,QB_role,...,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8,comp_pct,yds_per_attempt,td_pct,sack_pct,int_pct
1519,2024,2,Malik Willis,GB,IND,12.98,12.98,17.3875,17.6375,1.0,...,54.625000,0.791867,2.875000,14.500,0.125000,0.528302,5.452830,0.000000,0.226415,0.037736
1520,2024,2,Clayton Tune,ARI,LAR,-2.60,-1.60,20.4350,21.6850,2.0,...,8.833333,1.008426,1.333333,5.000,0.166667,0.571429,2.952381,0.000000,0.333333,0.095238
1521,2024,2,Jake Haener,NO,DAL,-0.30,-0.30,16.0650,16.6275,2.0,...,8.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1522,2024,2,Bryce Young,CAR,LAC,2.96,2.96,15.7100,15.8350,1.0,...,289.125000,0.656870,2.625000,15.875,0.125000,0.546218,5.693277,0.008403,0.126050,0.016807
1523,2024,2,Will Levis,TEN,NYJ,12.48,13.48,10.1425,10.6425,1.0,...,274.625000,0.664099,2.625000,10.000,0.125000,0.579909,6.552511,0.022831,0.114155,0.022831
1524,2024,2,C.J. Stroud,HOU,CHI,14.30,14.30,14.6450,15.7700,1.0,...,243.750000,0.953658,3.000000,13.125,0.125000,0.673913,7.952174,0.047826,0.086957,0.000000
1525,2024,2,Anthony Richardson,IND,GB,12.86,12.86,19.1225,20.6225,1.0,...,193.000000,1.389854,5.250000,38.000,1.000000,0.530303,8.575758,0.060606,0.075758,0.015152
1526,2024,2,Bo Nix,DEN,PIT,10.34,10.34,17.8950,18.3950,1.0,...,264.000000,0.522727,5.000000,35.000,1.000000,0.619048,3.285714,0.000000,0.047619,0.047619
1527,2024,2,Jayden Daniels,WAS,NYG,13.44,13.44,15.3075,16.3700,1.0,...,125.000000,1.472000,16.000000,88.000,2.000000,0.708333,7.666667,0.000000,0.083333,0.000000
1528,2024,2,Caleb Williams,CHI,HOU,9.36,9.36,19.4875,21.2375,1.0,...,214.000000,0.434579,5.000000,15.000,0.000000,0.482759,3.206897,0.000000,0.068966,0.000000


In [278]:
quarterback_df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [279]:
mean_dict = quarterback_df.mean(numeric_only=True).to_dict()

In [280]:
mean_dict

{'season': 2022.6788750817527,
 'week': 9.359058207979071,
 'FD_Pts': 14.126779184247537,
 'DK_Pts': 14.805752461322081,
 'FD_Pts_DvP': 17.154386505808755,
 'DK_Pts_DvP': 17.99350665453849,
 'QB_role': 1.2374100719424461,
 'completions_L8': 17.934557211373757,
 'attempts_L8': 27.858042137718396,
 'passing_yards_L8': 196.67526378896878,
 'passing_tds_L8': 1.1602329564919491,
 'interceptions_L8': 0.6268816375471051,
 'sacks_L8': 2.028618533744433,
 'passing_air_yards_L8': 216.09647567660159,
 'pacr_L8': 0.9656325151275502,
 'carries_L8': 3.660837615621788,
 'rushing_yards_L8': 16.073295649194932,
 'rushing_tds_L8': 0.1818876327509421,
 'comp_pct': 0.6373839519330623,
 'yds_per_attempt': 7.032624565253141,
 'td_pct': 0.0415801301294885,
 'sack_pct': 0.07775074760847986,
 'int_pct': 0.02364577218285631}

In [281]:
columns_to_fill = ['comp_pct', 'yds_per_attempt', 'td_pct', 'sack_pct', 'int_pct']

In [282]:
quarterback_df[columns_to_fill] = quarterback_df[columns_to_fill].apply(lambda col: col.fillna(col.mean()))

In [283]:
# quarterback_df.drop(columns = ['completions', 'attempts', 'passing_yards', 'passing_tds', 'sacks', 'interceptions',\
#                               'sack_fumbles_lost', 'pacr', 'carries', 'rushing_yards', 'rushing_tds', 'game_num'], inplace = True)

In [284]:
quarterback_df.tail(10)

,season,week,player_display_name,posteam,opponent_team,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP,QB_role,...,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8,comp_pct,yds_per_attempt,td_pct,sack_pct,int_pct
1519,2024,2,Malik Willis,GB,IND,12.98,12.98,17.3875,17.6375,1.0,...,54.625000,0.791867,2.875000,14.500,0.125000,0.528302,5.452830,0.000000,0.226415,0.037736
1520,2024,2,Clayton Tune,ARI,LAR,-2.60,-1.60,20.4350,21.6850,2.0,...,8.833333,1.008426,1.333333,5.000,0.166667,0.571429,2.952381,0.000000,0.333333,0.095238
1521,2024,2,Jake Haener,NO,DAL,-0.30,-0.30,16.0650,16.6275,2.0,...,8.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1522,2024,2,Bryce Young,CAR,LAC,2.96,2.96,15.7100,15.8350,1.0,...,289.125000,0.656870,2.625000,15.875,0.125000,0.546218,5.693277,0.008403,0.126050,0.016807
1523,2024,2,Will Levis,TEN,NYJ,12.48,13.48,10.1425,10.6425,1.0,...,274.625000,0.664099,2.625000,10.000,0.125000,0.579909,6.552511,0.022831,0.114155,0.022831
1524,2024,2,C.J. Stroud,HOU,CHI,14.30,14.30,14.6450,15.7700,1.0,...,243.750000,0.953658,3.000000,13.125,0.125000,0.673913,7.952174,0.047826,0.086957,0.000000
1525,2024,2,Anthony Richardson,IND,GB,12.86,12.86,19.1225,20.6225,1.0,...,193.000000,1.389854,5.250000,38.000,1.000000,0.530303,8.575758,0.060606,0.075758,0.015152
1526,2024,2,Bo Nix,DEN,PIT,10.34,10.34,17.8950,18.3950,1.0,...,264.000000,0.522727,5.000000,35.000,1.000000,0.619048,3.285714,0.000000,0.047619,0.047619
1527,2024,2,Jayden Daniels,WAS,NYG,13.44,13.44,15.3075,16.3700,1.0,...,125.000000,1.472000,16.000000,88.000,2.000000,0.708333,7.666667,0.000000,0.083333,0.000000
1528,2024,2,Caleb Williams,CHI,HOU,9.36,9.36,19.4875,21.2375,1.0,...,214.000000,0.434579,5.000000,15.000,0.000000,0.482759,3.206897,0.000000,0.068966,0.000000


# Wind speed, division games, odds
We'll bring in the odds CSVs. This is also where we can break off the data for the current week. We no longer need previous data to derive features.

In [285]:
fd_spreads = pd.read_csv('fd_spreads_' + str(season) + '_' + str(week) + '.csv')
dk_spreads = pd.read_csv('dk_spreads_' + str(season) + '_' + str(week) + '.csv')

In [286]:
fd_spreads.drop(columns = ['Unnamed: 0'], inplace = True)
dk_spreads.drop(columns = ['Unnamed: 0'], inplace = True)

In [287]:
fd_spreads.head()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
0,IND,CHI,43.5,0,0,0,-1.5,21.0,22.5,0,2024,3
1,CLE,NYG,38.5,1,1,0,-6.5,16.0,22.5,5,2024,3
2,TB,DEN,40.5,1,1,0,-6.5,17.0,23.5,5,2024,3
3,TEN,GB,38.5,1,0,0,-2.5,18.0,20.5,4,2024,3
4,MIN,HOU,45.5,0,1,0,1.5,23.5,22.0,0,2024,3


In [288]:
dk_spreads.head()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
0,IND,CHI,43.5,0,0,0,-1.0,21.25,22.25,0,2024,3
1,CLE,NYG,38.0,1,1,0,-6.5,15.75,22.25,5,2024,3
2,TB,DEN,41.0,1,1,0,-6.0,17.50,23.50,5,2024,3
3,TEN,GB,38.0,1,0,0,-3.0,17.50,20.50,4,2024,3
4,MIN,HOU,46.0,0,1,0,1.5,23.75,22.25,0,2024,3


In [289]:
quarterback_df.rename(columns = {'posteam': 'team', 'opponent_team': 'opponent'}, inplace = True)

In [290]:
quarterback_df = quarterback_df[(quarterback_df['season'] == season) & (quarterback_df['week'] == week)]

In [291]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107 entries, 1383 to 1489
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                107 non-null    int64  
 1   week                  107 non-null    int64  
 2   player_display_name   107 non-null    object 
 3   team                  107 non-null    object 
 4   opponent              107 non-null    object 
 5   FD_Pts                0 non-null      float64
 6   DK_Pts                0 non-null      float64
 7   FD_Pts_DvP            107 non-null    float64
 8   DK_Pts_DvP            107 non-null    float64
 9   QB_role               107 non-null    float64
 10  completions_L8        107 non-null    float64
 11  attempts_L8           107 non-null    float64
 12  passing_yards_L8      107 non-null    float64
 13  passing_tds_L8        107 non-null    float64
 14  interceptions_L8      107 non-null    float64
 15  sacks_L8              10

In [292]:
qb_df_fd = pd.merge(quarterback_df, fd_spreads, on = ['team', 'opponent', 'season', 'week'], how = 'left')
qb_df_dk = pd.merge(quarterback_df, dk_spreads, on = ['team', 'opponent', 'season', 'week'], how = 'left')

In [293]:
qb_df_fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                107 non-null    int64  
 1   week                  107 non-null    int64  
 2   player_display_name   107 non-null    object 
 3   team                  107 non-null    object 
 4   opponent              107 non-null    object 
 5   FD_Pts                0 non-null      float64
 6   DK_Pts                0 non-null      float64
 7   FD_Pts_DvP            107 non-null    float64
 8   DK_Pts_DvP            107 non-null    float64
 9   QB_role               107 non-null    float64
 10  completions_L8        107 non-null    float64
 11  attempts_L8           107 non-null    float64
 12  passing_yards_L8      107 non-null    float64
 13  passing_tds_L8        107 non-null    float64
 14  interceptions_L8      107 non-null    float64
 15  sacks_L8              1

In [294]:
divisions = {
    'CLE': 'AFC North',
    'LAR': 'NFC West',
    'LV': 'AFC West',
    'KC': 'AFC West',
    'CAR': 'NFC South',
    'NYG': 'NFC East',
    'HOU': 'AFC South',
    'DEN': 'AFC West',
    'MIN': 'NFC North',
    'TEN': 'AFC South',
    'JAX': 'AFC South',
    'SEA': 'NFC West',
    'DET': 'NFC North',
    'NO': 'NFC South',
    'CIN': 'AFC North',
    'ATL': 'NFC South',
    'NYJ': 'AFC East',
    'PHI': 'NFC East',
    'DAL': 'NFC East',
    'WAS': 'NFC East',
    'PIT': 'AFC North',
    'ARI': 'NFC West',
    'CHI': 'NFC North',
    'MIA': 'AFC East',
    'BUF': 'AFC East',
    'BAL': 'AFC North',
    'TB': 'NFC South',
    'SF': 'NFC West',
    'LAC': 'AFC West',
    'IND': 'AFC South',
    'GB': 'NFC North',
    'NE': 'AFC East'
}

# You now have a dictionary `divisions` where each team is mapped to its division.


# Adding div_game binary column

In [295]:
# Map the team and opponent columns to their respective divisions
qb_df_fd['team_division'] = qb_df_fd['team'].map(divisions)
qb_df_fd['opponent_division'] = qb_df_fd['opponent'].map(divisions)

# Create the div_game column (1 if they are in the same division, 0 otherwise)
qb_df_fd['div_game'] = np.where(qb_df_fd['team_division'] == qb_df_fd['opponent_division'], 1, 0)

# Optionally, drop the temporary division columns if you don't need them
qb_df_fd.drop(['team_division', 'opponent_division'], axis=1, inplace=True)


In [296]:
# Map the team and opponent columns to their respective divisions
qb_df_dk['team_division'] = qb_df_dk['team'].map(divisions)
qb_df_dk['opponent_division'] = qb_df_dk['opponent'].map(divisions)

# Create the div_game column (1 if they are in the same division, 0 otherwise)
qb_df_dk['div_game'] = np.where(qb_df_dk['team_division'] == qb_df_dk['opponent_division'], 1, 0)

# Optionally, drop the temporary division columns if you don't need them
qb_df_dk.drop(['team_division', 'opponent_division'], axis=1, inplace=True)

In [297]:
qb_df_fd.rename(columns = {'player_display_name': 'name'}, inplace = True)
qb_df_dk.rename(columns = {'player_display_name': 'name'}, inplace = True)

In [298]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week
4,107027-63115,Lamar Jackson,QB,8800,BAL,DAL,0,Active,09-22-2024,3
5,107027-57439,Patrick Mahomes,QB,8700,KC,ATL,0,Active,09-22-2024,3
9,107027-69531,Jalen Hurts,QB,8500,PHI,NO,0,Active,09-22-2024,3
13,107027-63484,Kyler Murray,QB,8300,ARI,DET,1,Active,09-22-2024,3
15,107027-129471,C.J. Stroud,QB,8200,HOU,MIN,0,Active,09-22-2024,3


In [299]:
draftkings_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week
10,35939904,Lamar Jackson,QB,7500,BAL,DAL,0,Active,09-22-2024,3
14,35939905,Jalen Hurts,QB,7300,PHI,NO,0,Active,09-22-2024,3
16,35939906,C.J. Stroud,QB,7100,HOU,MIN,0,Active,09-22-2024,3
23,35939907,Kyler Murray,QB,6900,ARI,DET,1,Active,09-22-2024,3
26,35939908,Anthony Richardson,QB,6700,IND,CHI,1,Active,09-22-2024,3


In [300]:
fanduel_df.rename(columns = {'player_display_name': 'name'}, inplace = True)
draftkings_df.rename(columns = {'player_display_name': 'name'}, inplace = True)

In [301]:
# starting_qbs = [('BUF', 'Josh Allen'), ('MIA', 'Skylar Thompson'), ('NYJ', 'Aaron Rodgers'), ('NE', 'Jacoby Brissett'),\
#                ('BAL', 'Lamar Jackson'), ('CIN', 'Joe Burrow'), ('CLE', 'Deshaun Watson'), ('PIT', 'Justin Fields'),\
#                ('IND', 'Anthony Richardson'), ('JAX', 'Trevor Lawrence'), ('HOU', 'C.J. Stroud'), ('TEN', 'Will Levis'),\
#                ('KC', 'Patrick Mahomes'), ('LV', 'Gardner Minshew'), ('LAC', 'Justin Herbert'), ('DEN', 'Bo Nix'),\
#                ('DAL', 'Dak Prescott'), ('PHI', 'Jalen Hurts'), ('NYG', 'Daniel Jones'), ('WAS', 'Jayden Daniels'),\
#                ('MIN', 'Sam Darnold'), ('DET', 'Jared Goff'), ('GB', 'Malik Willis'), ('CHI', 'Caleb Williams'),\
#                ('TB', 'Baker Mayfield'), ('NO', 'Derek Carr'), ('CAR', 'Bryce Young'), ('ATL', 'Kirk Cousins'),\
#                ('SF', 'Brock Purdy'), ('SEA', 'Geno Smith'), ('LAR', 'Matthew Stafford'), ('ARI', 'Kyler Murray')]

In [302]:
# starting_qbs = pd.DataFrame(data = starting_qbs, columns = ['team', 'name'])

In [303]:
# starting_qbs.to_csv('starting_QBs.csv', index = False)

In [304]:
qb_1 = pd.read_csv('starting_QBs.csv')

In [305]:
qb_df_fd.loc[
    qb_df_fd.set_index(['team', 'name']).index.isin(qb_1.set_index(['team', 'name']).index), 
    'QB_role'
] = 1

In [306]:
qb_df_dk.loc[
    qb_df_dk.set_index(['team', 'name']).index.isin(qb_1.set_index(['team', 'name']).index), 
    'QB_role'
] = 1

In [307]:
features = ['FD_Pts_DvP', 'DK_Pts_DvP', 'QB_role', 'completions_L8', 'attempts_L8',
       'passing_yards_L8', 'passing_tds_L8', 'interceptions_L8', 'sacks_L8',
       'passing_air_yards_L8', 'pacr_L8', 'carries_L8', 'rushing_yards_L8',
       'rushing_tds_L8', 'comp_pct', 'yds_per_attempt', 'td_pct', 'sack_pct',
       'int_pct', 'wind', 'div_game', 'spread_line', 'total_line', 'outdoors',
       'grass', 'home_team', 'pred_total', 'opp_total']

In [308]:
qb_df_fd.drop(columns = ['FD_Pts', 'DK_Pts'], inplace = True)
qb_df_dk.drop(columns = ['FD_Pts', 'DK_Pts'], inplace = True)

In [309]:
qb_df_fd.replace([np.inf, -np.inf], np.nan, inplace=True)
qb_df_dk.replace([np.inf, -np.inf], np.nan, inplace=True)

In [310]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week
4,107027-63115,Lamar Jackson,QB,8800,BAL,DAL,0,Active,09-22-2024,3
5,107027-57439,Patrick Mahomes,QB,8700,KC,ATL,0,Active,09-22-2024,3
9,107027-69531,Jalen Hurts,QB,8500,PHI,NO,0,Active,09-22-2024,3
13,107027-63484,Kyler Murray,QB,8300,ARI,DET,1,Active,09-22-2024,3
15,107027-129471,C.J. Stroud,QB,8200,HOU,MIN,0,Active,09-22-2024,3


In [311]:
fanduel_df = fanduel_df[['name', 'position', 'salary', 'team', 'opponent', 'status', 'week']]
draftkings_df = draftkings_df[['name', 'position', 'salary', 'team', 'opponent', 'status', 'week']]

In [312]:
fanduel_df.head()

,name,position,salary,team,opponent,status,week
4,Lamar Jackson,QB,8800,BAL,DAL,Active,3
5,Patrick Mahomes,QB,8700,KC,ATL,Active,3
9,Jalen Hurts,QB,8500,PHI,NO,Active,3
13,Kyler Murray,QB,8300,ARI,DET,Active,3
15,C.J. Stroud,QB,8200,HOU,MIN,Active,3


In [313]:
draftkings_df.head()

,name,position,salary,team,opponent,status,week
10,Lamar Jackson,QB,7500,BAL,DAL,Active,3
14,Jalen Hurts,QB,7300,PHI,NO,Active,3
16,C.J. Stroud,QB,7100,HOU,MIN,Active,3
23,Kyler Murray,QB,6900,ARI,DET,Active,3
26,Anthony Richardson,QB,6700,IND,CHI,Active,3


# Merging player list with the model features
We need to bring back the salary and status columns. This also will filter out players who aren't on the list for one site or the other.

In [314]:
qb_df_fd = pd.merge(fanduel_df, qb_df_fd, on = ['name', 'team', 'opponent', 'week'], how = 'left')

In [315]:
qb_df_dk = pd.merge(draftkings_df, qb_df_dk, on = ['name', 'team', 'opponent', 'week'], how = 'left')

In [316]:
qb_df_fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  107 non-null    object 
 1   position              107 non-null    object 
 2   salary                107 non-null    int64  
 3   team                  107 non-null    object 
 4   opponent              107 non-null    object 
 5   status                107 non-null    object 
 6   week                  107 non-null    int64  
 7   season                107 non-null    int64  
 8   FD_Pts_DvP            107 non-null    float64
 9   DK_Pts_DvP            107 non-null    float64
 10  QB_role               107 non-null    float64
 11  completions_L8        107 non-null    float64
 12  attempts_L8           107 non-null    float64
 13  passing_yards_L8      107 non-null    float64
 14  passing_tds_L8        107 non-null    float64
 15  interceptions_L8      1

In [317]:
qb_df_dk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  77 non-null     object 
 1   position              77 non-null     object 
 2   salary                77 non-null     int64  
 3   team                  77 non-null     object 
 4   opponent              77 non-null     object 
 5   status                77 non-null     object 
 6   week                  77 non-null     int64  
 7   season                77 non-null     int64  
 8   FD_Pts_DvP            77 non-null     float64
 9   DK_Pts_DvP            77 non-null     float64
 10  QB_role               77 non-null     float64
 11  completions_L8        77 non-null     float64
 12  attempts_L8           77 non-null     float64
 13  passing_yards_L8      77 non-null     float64
 14  passing_tds_L8        77 non-null     float64
 15  interceptions_L8      77 

In [318]:
qb_df_fd = qb_df_fd.set_index(['name', 'team', 'position', 'salary', 'opponent', 'status', 'season', 'week'], drop = True)
qb_df_dk = qb_df_dk.set_index(['name', 'team', 'position', 'salary', 'opponent', 'status', 'season', 'week'], drop = True)

In [319]:
qb_df_fd = qb_df_fd[features]
qb_df_dk = qb_df_dk[features]

In [320]:
qb_df_fd.to_csv('FD_qb_for_model_' + str(season) + '_' + str(week) + '.csv')
qb_df_dk.to_csv('DK_qb_for_model_' + str(season) + '_' + str(week) + '.csv')

In [321]:
###FANDUEL SCORING
#Rushing yards made = 0.1pts	
#Rushing touchdowns = 6pts	
#Passing yards = 0.04pts	
#Passing touchdowns = 4pts	
#Interceptions = -1pt	
#Receiving yards = 0.1pts	
#Receiving touchdowns = 6pts	
#Receptions = 0.5pts	
#Kickoff return touchdowns = 6pts	
#Punt return touchdowns = 6pts	
#Fumbles lost = -2pts	
#Own fumbles recovered touchdowns = 6pts	
#Two-point conversions scored = 2pts	
#Two-point conversion passes = 2pts	
#Field-goals from 0-39 yards = 3pts	
#Field-goals from 40-49 yards = 4pts	
#Field-goals from 50+ yards = 5pts	
#Extra-point conversions = 1pt

###DRAFTKINGS SCORING
#PAssing TD = 4 pts
#passing yards = .04 pts
#300 passing yards = 3 pts (bonus)
#Interception = -1 pts
#Rushing TD = 6 pts
#Rushing yds = 0.1 pts
#100 yd rushing game = 3 pts (bonus)
#Receiving TD = 6 pts
#Receiving yds = 0.1 pts
#100 receiving yards game = 3 pts (bonus)
#Receptions = 1 pt
#Punt/kickoff/FG return for TD = 6 pts
#Fumble lost = -1 pt
#2 pt conversion (pass, run or catch) = 2 pts
#Offensive fumble recovery TD = 6

In [322]:
# #import sqlite3

# # Establish a connection to the SQLite database
# conn = sqlite3.connect('nfl_dfs.db')

# # Define the table name
# table_name = 'qb_dataset'

# # Specify data types
# dtype = {
#     'season': 'INTEGER',
#     'week': 'INTEGER',
#     'player_id': 'TEXT',
#     'player_display_name': 'TEXT',
#     'team': 'TEXT',
#     'FD_Pts': 'REAL',
#     'DK_Pts': 'REAL',
#     'FD_Pts_DvP': 'REAL',
#     'DK_Pts_DvP': 'REAL',
#     'QB_role': 'REAL',
#     'completions_L8': 'REAL',
#     'attempts_L8': 'REAL',
#     'passing_yards_L8': 'REAL',
#     'passing_tds_L8': 'REAL',
#     'interceptions_L8': 'REAL',
#     'sacks_L8': 'REAL',
#     'passing_air_yards_L8': 'REAL',
#     'pacr_L8': 'REAL',
#     'carries_L8': 'REAL',
#     'rushing_yards_L8': 'REAL',
#     'rushing_tds_L8': 'REAL',
#     'comp_pct': 'REAL',
#     'yds_per_attempt': 'REAL',
#     'td_pct': 'REAL',
#     'sack_pct': 'REAL',
#     'int_pct': 'REAL',
#     'wind': 'REAL',
#     'div_game': 'REAL',
#     'opponent': 'TEXT',
#     'spread_line': 'REAL',
#     'total_line': 'REAL',
#     'outdoors': 'INTEGER',
#     'grass': 'INTEGER',
#     'home_team': 'INTEGER',
#     'pred_total': 'REAL',
#     'opp_total': 'REAL'
# }

# # Write the DataFrame to the SQLite table
# quarterback_df.to_sql(table_name, conn, if_exists='replace', index=False, dtype=dtype)

# # Confirm that the data has been written
# print(f"Data written to table {table_name} in SQLite database nfl_dfs.db")

# # Close the connection
# conn.close()
